In [1]:
import pandas as pd
import numpy as np
import time_series_transform.base as base

In [2]:
arr= np.array([1,1,3,4,5,6,7])
fact = base.Time_Series_Tensor_Factory(arr,'sequence')

In [3]:
time = fact.get_time_series_tensor('time',2,outType = np.float32)

In [4]:
fb = fact.get_time_series_tensor('time',2,outType = np.float32)

In [8]:
label = base.Time_Series_Tensor_Factory(arr,'label')
label =  label.get_time_series_tensor('label',2,outType = np.float32)

In [9]:
cate = [1,2,3]
cate = base.Time_Series_Tensor_Factory(cate,'category')
cate =  cate.get_time_series_tensor('category',2,batchSize= label.get_data_shape()[0],outType = np.float32)

In [10]:
dataset = base.Time_Series_Dataset([time,label,cate])

In [11]:
dataset.make_dataset()

{'time': array([[[1, 1],
         [1, 1]],
 
        [[1, 1],
         [3, 3]],
 
        [[3, 3],
         [4, 4]],
 
        [[4, 4],
         [5, 5]],
 
        [[5, 5],
         [6, 6]]]),
 'label': array([3, 4, 5, 6, 7]),
 'category': array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])}